# Depth-of-field with ParaView path-tracing

A demonstration created by Jean M. Favre, Swiss National Supercomputing Center.

Tested on Piz Daint, with ParaView 5.10.1
Tue Mar 15 02:51:54 PM CET 2022

In [ ]:
from paraview.simple import *
print("Using ", GetParaViewSourceVersion())
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

mf1 = "/users/jfavre/Projects/ParaView/ospray_mats.json"
materialLibrary1 = GetMaterialLibrary()
print("using materials: {:}".format(mf1))
materialLibrary1.LoadMaterials = mf1

In [ ]:
ProgressivePasses = 0
SamplesPerPixel = 1

In [ ]:
# Create a new 'Render View'
renderView1 = GetRenderView()
renderView1.AxesGrid = 'GridAxes3DActor'
renderView1.OrientationAxesVisibility = 0
renderView1.CenterOfRotation = [0.0, 3.0985385850071907, -0.1246500015258789]
renderView1.StereoType = 0
renderView1.CameraPosition = [-24.562376867580436, 17.193587915885082, 6.724729205257313]
renderView1.CameraFocalPoint = [-8.328268490755832e-19, 3.0985385850071907, -0.12465000152587886]
renderView1.CameraViewUp = [0.46664201880300193, 0.8751921424967176, -0.12760854203173097]
renderView1.CameraParallelScale = 7.615965180894449
renderView1.Background = [0.32, 0.34, 0.43]
renderView1.EnableRayTracing = 0 # Classic OpenGL
renderView1.SamplesPerPixel = SamplesPerPixel
renderView1.OSPRayMaterialLibrary = materialLibrary1

In [ ]:
renderView1.ViewSize = [600, 400]

In [ ]:
from paraview.modules.vtkRemotingViews import vtkPVOpenGLInformation

info = vtkPVOpenGLInformation()
info.CopyFromObject(None)
print("Vendor:   %s" % info.GetVendor())
print("Version:  %s" % info.GetVersion())
print("Renderer: %s" % info.GetRenderer())

In [ ]:
teapot_lid = PLYReader(FileNames=['../Data/teapot_lid.ply'])

teapot_body = PLYReader(FileNames=['../Data/teapot_body.ply'])

# make two more copies of the teapot+lid, which we will move in space in the X direction
teapot1 = PassArrays(Input=teapot_body)
teapot2 = PassArrays(Input=teapot_body)

lid1 = PassArrays(Input=teapot_lid)
lid2 = PassArrays(Input=teapot_lid)

teapot0Display = Show(teapot_body)
teapot0Display.Representation = 'Surface'
teapot0Display.ColorArrayName = [None, '']
teapot0Display.OSPRayMaterial = 'glass'

teapot1Display = Show(teapot1)
teapot1Display.Representation = 'Surface'
teapot1Display.ColorArrayName = [None, '']
teapot1Display.OSPRayMaterial = 'glass'
teapot1Display.Position = [15, 0, 0]

teapot2Display = Show(teapot2)
teapot2Display.Representation = 'Surface'
teapot2Display.ColorArrayName = [None, '']
teapot2Display.OSPRayMaterial = 'glass'
teapot2Display.Position = [-10, 0, 0]

lid0Display = Show(teapot_lid)
lid0Display.Representation = 'Surface'
lid0Display.ColorArrayName = [None, '']
lid0Display.OSPRayMaterial = 'greenish glass'

lid1Display = Show(lid1)
lid1Display.Representation = 'Surface'
lid1Display.ColorArrayName = [None, '']
lid1Display.OSPRayMaterial = 'greenish glass'
lid1Display.Position = [15, 0, 0]

lid2Display = Show(lid2)
lid2Display.Representation = 'Surface'
lid2Display.ColorArrayName = [None, '']
lid2Display.OSPRayMaterial = 'greenish glass'
lid2Display.Position = [-10, 0, 0]

In [ ]:
plane1 = Plane()
plane1.Origin = [-40.0, 0.0, -30.0]
plane1.Point1 = [80.0, 0.0, -30.0]
plane1.Point2 = [-40.0, 0.0, 30.0]
plane1.XResolution = 20
plane1.YResolution = 20

In [ ]:
plane1Display = Show(plane1, renderView1)

# trace defaults for the display properties.
plane1Display.Representation = 'Surface'
plane1Display.ColorArrayName = [None, '']
plane1Display.OSPRayMaterial = 'checker alpha'

In [ ]:
# import the PVDisplay widget, then instantiate one for renV

from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(renderView1)

# show the widget once
display(pvdisp)

In [ ]:
# Third do OSPRay rendering without shadows
renderView1.EnableRayTracing = 1
renderView1.BackEnd = 'OSPRay raycaster'
renderView1.LightScale = 1
renderView1.Shadows = 0
renderView1.AmbientSamples = 1
renderView1.Denoise = 1
renderView1.ProgressivePasses = 0  # must launch with --enable-streaming

In [ ]:
# Fourth do OSPRay rendering with shadows
renderView1.Shadows = 1

In [ ]:
# Fifth do OSPRay rendering with *soft* shadows
renderView1.BackEnd = 'OSPRay pathtracer'

## Make three different rendering with depth-of-field
we will move the center of attention to each teapot, one at a time

In [ ]:
camera=GetActiveCamera()
print("camera.GetFocalDistance()=", camera.GetFocalDistance())
print("camera.GetFocalDisk()=", camera.GetFocalDisk())
SamplesPerPixel = 10
renderView1.SamplesPerPixel = SamplesPerPixel

In [ ]:
# focus on the far-away teapot
camera.SetFocalDistance(39)
camera.SetFocalDisk(2)

In [ ]:
# focus on the teapot in the center
camera.SetFocalDistance(29)
camera.SetFocalDisk(2)

In [ ]:
# focus on the closest teapot
camera.SetFocalDistance(14)
camera.SetFocalDisk(.2)

In [ ]:
from ipywidgets import interact

def DepthOfField(distance, focalDisk):
    camera.SetFocalDistance(distance)
    camera.SetFocalDisk(focalDisk)

interact(DepthOfField, distance=(1,50), focalDisk=(0.,3.))